In [1]:
import simdna

In [10]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
#@title
from __future__ import absolute_import, division, print_function
from collections import OrderedDict
import re
import gzip
import os
import json
from simdna import random


DEFAULT_LETTER_TO_INDEX = {'A': 0, 'C': 1, 'G': 2, 'T': 3}


DEFAULT_BACKGROUND_FREQ = OrderedDict(
    [('A', 0.3), ('C', 0.2), ('G', 0.2), ('T', 0.3)])


DEFAULT_DINUC_FREQ = OrderedDict([
 ('AA',0.095),
 ('AC',0.050),
 ('AG',0.071),
 ('AT',0.075),
 ('CA',0.073),
 ('CC',0.054),
 ('CG',0.010),
 ('CT',0.072),
 ('GA',0.060),
 ('GC',0.044),
 ('GG',0.054),
 ('GT',0.050),
 ('TA',0.064),
 ('TC',0.060),
 ('TG',0.073),
 ('TT',0.095),
])


def get_file_handle(filename,mode="r"):
    if (re.search('.gz$',filename) or re.search('.gzip',filename)):
        if (mode=="r"):
            mode="rb";
        elif (mode=="w"):
            #I think write will actually append if the file already
            #exists...so you want to remove it if it exists
            if os.path.isfile(filename):
                os.remove(filename);
        return gzip.open(filename,mode)
    else:
        return open(filename,mode) 


def default_tab_seppd(s):
    s = trim_newline(s)
    s = split_by_delimiter(s, "\t")
    return s


def trim_newline(s):
    return s.rstrip('\r\n')


def split_by_delimiter(s, delimiter):
    return s.split(delimiter)


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
        line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();


def process_line(line, i, ignore_input_title,
                 transformation, action, progress_update=None):
    if (i > 1 or (ignore_input_title==False)):
        action(transformation(line),i)


def print_progress(progress_update, i, file_name=None):
    if progress_update is not None:
        if (i%progress_update == 0):
            print ("Processed "+str(i)+" lines"
                   +str("" if file_name is None else " of "+file_name))


class VariableWrapper():
    """ For when I want reference-type access to an immutable"""
    def __init__(self, var):
        self.var = var   


def enum(**enums):
    class Enum(object):
        pass
    to_return = Enum
    for key,val in enums.items():
        if hasattr(val, '__call__'): 
            setattr(to_return, key, staticmethod(val))
        else:
            setattr(to_return, key, val)
    to_return.vals = [x for x in enums.values()]
    to_return.the_dict = enums
    return to_return


def combine_enums(*enums):
    new_enum_dict = OrderedDict()
    for an_enum in enums:
        new_enum_dict.update(an_enum.the_dict)
    return enum(**new_enum_dict)


def sampleFromProbsArr(arrWithProbs):
    """Samples from a discrete distribution.
    Arguments:
        arrWithProbs: array of probabilities
    Returns:
        an index, sampled with the probability of that index in
    array of probabilities.
    """
    randNum = random.random()
    cdfSoFar = 0
    for (idx, prob) in enumerate(arrWithProbs):
        cdfSoFar += prob
        if (cdfSoFar >= randNum or idx == (len(arrWithProbs) - 1)):  # need the
            # letterIdx==(len(row)-1) clause because of potential floating point errors
            # that mean arrWithProbs doesn't sum to 1
            return idx


reverseComplementLookup = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G',
                           'a': 't', 't': 'a', 'g': 'c', 'c': 'g', 'N': 'N', 'n': 'n'}


def reverseComplement(sequence):
    reversedSequence = sequence[::-1]
    reverseComplemented = "".join(
        [reverseComplementLookup[x] for x in reversedSequence])
    return reverseComplemented


def sampleWithoutReplacement(arr, numToSample):
    arrayCopy = [x for x in arr]
    for i in range(numToSample):
        randomIndex = int(random.random() * (len(arrayCopy) - i)) + i
        swapIndices(arrayCopy, i, randomIndex)
    return arrayCopy[0:numToSample]


def swapIndices(arr, idx1, idx2):
    temp = arr[idx1]
    arr[idx1] = arr[idx2]
    arr[idx2] = temp


def get_file_name_parts(file_name):
    p = re.compile(r"^(.*/)?([^\./]+)(\.[^/]*)?$")
    m = p.search(file_name)
    return FileNameParts(m.group(1), m.group(2), m.group(3))


class FileNameParts(object):

    def __init__(self, directory, core_file_name, extension):
        self.directory = directory if (directory is not None) else os.getcwd()
        self.core_file_name = core_file_name
        self.extension = extension

    def get_full_path(self):
        return self.directory+"/"+self.file_name

    def get_core_file_name_and_extension(self):
        return self.core_file_name+self.extension

    def get_transformed_core_file_name(self, transformation, extension=None):
        to_return = transformation(self.core_file_name)
        if (extension is not None):
            to_return = to_return + extension
        else:
            if (self.extension is not None):
                to_return = to_return + self.extension
        return to_return

    def get_transformed_file_path(self, transformation, extension=None):
        return (self.directory+"/"+
                self.get_transformed_core_file_name(transformation,
                                                    extension=extension))


def format_as_json(jsonable_data):
    return json.dumps(jsonable_data, indent=4, separators=(',', ': '))


class ArgumentToAdd(object):
    """
        Class to append runtime arguments to a string
        to facilitate auto-generation of output file names.
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = (','.join([str(el) for el in self.val])\
                   if (isinstance(self.val, str)==False and
                       hasattr(self.val,"__len__")) else str(self.val))
        return self.argNamePrefix()+string;
        # return self.argNamePrefix()+str(self.val).replace(".","p");


class FloatArgument(ArgumentToAdd):
    """
       Replace the period with a p 
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = str(self.val)
        string = string.replace(".","p")
        return self.argNamePrefix()+string


class BooleanArgument(ArgumentToAdd):

    def transform(self):
        assert self.val  # should be True if you're calling transformation
        return self.argumentName


class CoreFileNameArgument(ArgumentToAdd):

    def transform(self):
        import fileProcessing as fp
        return self.argNamePrefix() + fp.getCoreFileName(self.val)


class ArrArgument(ArgumentToAdd):

    def __init__(self, val, argumentName, sep="+", toStringFunc=str):
        super(ArrArgument, self).__init__(val, argumentName)
        self.sep = sep
        self.toStringFunc = toStringFunc

    def transform(self):
        return self.argNamePrefix() + self.sep.join([self.toStringFunc(x) for x in self.val])


class ArrOfFileNamesArgument(ArrArgument):

    def __init__(self, val, argumentName, sep="+"):
        import fileProcessing as fp
        super(ArrOfFileNamesArgument, self).__init__(val, argumentName,
                                                     sep, toStringFunc=lambda x: fp.getCoreFileName(x))


def addArguments(string, args, joiner="_"):
    """
        args is an array of ArgumentToAdd.
    """
    for arg in args:
        string = string + ("" if arg.val is None or arg.val is False or (hasattr(
            arg.val, "__len__") and len(arg.val) == 0) else joiner + arg.transform())
    return string
class Embedding(object):
    """Represents something that has been embedded in a sequence.
    Think of this as a combination of an embeddable + a start position.
    Arguments:
        what: object representing the thing that has been embedded.\
            Should have`` __str__`` and ``__len__`` defined.\
            Often is an instance of :class:`.AbstractEmbeddable`
        startPos: int, the position relative to the start of the parent\
            sequence at which seq has been embedded
    """

    def __init__(self, what, startPos):
        self.what = what
        self.startPos = startPos

    def __str__(self):
        return "pos-" + str(self.startPos) + "_" + str(self.what)

    @classmethod
    def fromString(cls, string, whatClass=None):
        """Recreate an :class:`.Embedding` object from a string.
        Arguments:
            string: assumed to have format:\
                ``description[-|_]startPos[-|_]whatString``, where
                ``whatString`` will be provided to ``whatClass``
            whatClass: the class (usually a :class:`.AbstractEmbeddable`) that\
                will be used to instantiate the what from the whatString
        Returns:
            The Embedding class called with
            ``what=whatClass.fromString(whatString)`` and
            ``startPos=int(startPos)``
        """
        if (whatClass is None):
            from simdna.synthetic.embeddables import StringEmbeddable
            whatClass = StringEmbeddable
        # was printed out as pos-[startPos]_[what], but the
        #[what] may contain underscores, hence the maxsplit
        # to avoid splitting on them.
        p = re.compile(r"pos\-(\d+)_(.*)$")
        m = p.search(string)
        startPos = m.group(1)
        whatString = m.group(2) 
        return cls(what=whatClass.fromString(whatString),
                   startPos=int(startPos))
      
def getEmbeddingsFromString(string):
    """Get a series of :class:`.Embedding` objects from a string.
    
    Splits the string on commas, and then passes the comma-separated vals
        to :func:`.Embedding.fromString`
    Arguments:
        string: The string to turn into an array of Embedding objects
    Returns:
        an array of :class:`.Embedding` objects
    """
    if len(string) == 0:
        return []
    else:
        embeddingStrings = string.split(",")
        return [Embedding.fromString(x) for x in embeddingStrings]
      
def read_simdata_file(simdata_file, one_hot_encode=False, ids_to_load=None):
    ids = []
    sequences = []
    embeddings = []
    labels = []
    if (ids_to_load is not None):
        ids_to_load = set(ids_to_load)
    def action(inp, line_number):
        if (line_number > 1):
            if (ids_to_load is None or (inp[0] in ids_to_load)):
                ids.append(inp[0]) 
                sequences.append(inp[1])
                embeddings.append(getEmbeddingsFromString(inp[2]))
                labels.append([int(x) for x in inp[3:]])
    perform_action_on_each_line_of_file(
        file_handle=get_file_handle(simdata_file),
        action=action,
        transformation=default_tab_seppd)
    return enum(
            ids=ids,
            sequences=sequences,
            embeddings=embeddings,
            labels=np.array(labels))
  


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
#         line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();

In [6]:
import keras 
import keras_genomics
import numpy as np

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.engine import Layer
from keras.models import Sequential 
import keras.layers as k1
from keras.engine.base_layer import InputSpec

from sklearn.model_selection import train_test_split

In [7]:
# !motif_density_and_position_sim.py --seed 1234 --motifNames GATA_disc1 TAL1_known1 --max-motifs 4 --min-motifs 1 --mean-motifs 2 --zero-prob 0.5 --seqLength 1000 --posSdevInBp 100 --numSeqs 20000

In [8]:
# !head DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-20000.simdata

In [7]:
sequences = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").sequences
embeddings = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").embeddings

In [8]:
def check(embeddings): 
    if len(embeddings) > 0:
        all_chars = ""
        for x in embeddings: 
            all_chars += str(x.what)
        if 'TAL' in all_chars and 'GATA' in all_chars: 
            return [1, 1]
        if 'TAL' in all_chars and 'GATA' not in all_chars:
            return [1, 0]
        if 'TAL' not in all_chars and 'GATA' in all_chars:
            return [0, 1]
    return [0,0]

In [9]:
y= np.zeros((0,2))
for x in embeddings: 
    thing = check(x)
    y = np.append(y, [thing], axis=0)

In [10]:
import numpy as np

# def mutate(s, num, target):
#     change_locs = set(sample(range(len(s)), num))
#     changed = (target if i in change_locs else c for i,c in enumerate(s))
#     return ''.join(changed)

def mutate(y, mutation_prob):
    np.random.seed(1234)
    mutated_y = []
    for row in y:
        new_labels_for_row = []
        for label in row:
            if np.random.uniform() < mutation_prob:
                new_labels_for_row.append(1-label)
            else:
                new_labels_for_row.append(label)
        mutated_y.append(new_labels_for_row)
    return np.array(mutated_y)

In [11]:
ltrdict = {'a':[1,0,0,0],
           'c':[0,1,0,0],
           'g':[0,0,1,0],
           't':[0,0,0,1],
           'n':[0,0,0,0],
           'A':[1,0,0,0],
           'C':[0,1,0,0],
           'G':[0,0,1,0],
           'T':[0,0,0,1],
           'N':[0,0,0,0]}

def onehot_encode(sequence):
    return np.array([ltrdict[x] for x in sequence])

x = np.array([onehot_encode(x) for x in sequences]) 

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [13]:
x_test[0][1]

array([0, 0, 0, 1])

In [14]:
noisy_y_train = mutate(y_train, mutation_prob=0.2)

In [15]:
x_train_augment = np.asarray([val for val in x_train for __ in (0,1)])
for i in range(len(x_train_augment)):
    if i % 2 == 1:
        x_train_augment[i] = np.flip(x_train_augment[i])

x_test_augment = np.asarray([val for val in x_test for __ in (0,1)])
for i in range(len(x_test_augment)):
    if i % 2 == 1:
        x_test_augment[i] = np.flip(x_test_augment[i])

In [16]:
noisy_y_train_augment = np.asarray([val for val in noisy_y_train for __ in (0,1)])
y_test_augment = np.asarray([val for val in y_test for __ in (0,1)])

In [17]:
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))


In [18]:
kernel_size = 15
filters= 30
input_length = 1000

In [19]:
seed_num = 1000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

seed(seed_num)
set_random_seed(seed_num)

In [20]:
from keras.layers import Input
from keras.models import Model
from keras.models import load_model

In [2]:
filename = ('Seed %s.txt' % seed_num, str(seed_num))[0]
f = open(filename, 'w')

NameError: name 'seed_num' is not defined

In [28]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

def evaluate(model, x, y): 
    y_pred = model.predict(x)
    auroc = roc_auc_score(y, model.predict(x)) 
    auprc = average_precision_score(y, model.predict(x))
    print("auroc: " + str(auroc))
    print("auprc: "+ str(auprc))
    f.write("auroc: " + str(auroc) + "\n")
    f.write("auprc: " + str(auprc) + "\n")

In [30]:
class RevCompSpatialDropout1D(Dropout): 
    def __init__(self, rate,**kwargs): 
        super(RevCompSpatialDropout1D, self).__init__(rate, **kwargs)
        self.seed = 3
        self.input_spec = InputSpec(ndim = 3)

    def _get_noise_shape(self, inputs): 
        input_shape = K.shape(inputs)
        noise_shape = (input_shape[0], 1, 1, int(self.num_input_chan/2)) 
        return noise_shape
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        self.input_len = input_shape[1]
        super(RevCompSpatialDropout1D, self).build(input_shape)

    def call(self, inputs, training=None): 
        inputs_fwdandrevconcat = K.concatenate(
                tensors = [
                    inputs[:,:,None,:int(self.num_input_chan/2)],
                    inputs[:,:,None,int(self.num_input_chan/2):][:,:,:,::-1]],
                axis=2)

        if 0. < self.rate < 1.: 
            noise_shape = self._get_noise_shape(inputs)
            def dropped_inputs(): 
                dropped = K.dropout(inputs_fwdandrevconcat,
                                    self.rate, noise_shape, seed = self.seed)
                dropped = K.reshape(dropped, (-1, int(self.input_len), int(self.num_input_chan)))
                return K.concatenate(
                    tensors = [
                        dropped[:,:,:int(self.num_input_chan/2)],
                        dropped[:,:,int(self.num_input_chan/2):][:,:,::-1]],
                    axis=-1)

            return K.in_train_phase(dropped_inputs, inputs, training = training)

        return inputs

In [44]:
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
import numbers
from tensorflow.python.framework import tensor_util

import keras
from keras.initializers import Constant
import keras.backend as K
from keras.engine.topology import Layer
import tensorflow as tf

def _get_noise_shape(x, noise_shape):
  # If noise_shape is none return immediately.
  if noise_shape is None:
    return array_ops.shape(x)

  try:
    # Best effort to figure out the intended shape.
    # If not possible, let the op to handle it.
    # In eager mode exception will show up.
    noise_shape_ = tensor_shape.as_shape(noise_shape)
  except (TypeError, ValueError):
    return noise_shape

  if x.shape.dims is not None and len(x.shape.dims) == len(noise_shape_.dims):
    new_dims = []
    for i, dim in enumerate(x.shape.dims):
      if noise_shape_.dims[i].value is None and dim.value is not None:
        new_dims.append(dim.value)
      else:
        new_dims.append(noise_shape_.dims[i].value)
    return tensor_shape.TensorShape(new_dims)

  return noise_shape

class MCRCDropout(Layer):
    """Applies MC Dropout to the input.
       The applied noise vector is symmetric to reverse complement symmetry
       Class structure only slightly adapted 
    Dropout consists in randomly setting
    a fraction `rate` of input units to 0 at each update during training time,
    which helps prevent overfitting.
    Remains active ative at test time so sampling is required
    # Arguments
        rate: float between 0 and 1. Fraction of the input units to drop.
        noise_shape: 1D integer tensor representing the shape of the
            binary dropout mask that will be multiplied with the input.
            For instance, if your inputs have shape
            `(batch_size, timesteps, features)` and
            you want the dropout mask to be the same for all timesteps,
            you can use `noise_shape=(batch_size, 1, features)`.
        seed: A Python integer to use as random seed.
    # References
        - [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)
    """
    def __init__(self, rate, noise_shape=None, seed=None, **kwargs):
        super(MCRCDropout, self).__init__(**kwargs)
        self.rate = min(1., max(0., rate))
        self.noise_shape = noise_shape
        self.seed = seed
        self.supports_masking = True

    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape

        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            import numpy as np
            noise_shape = self._get_noise_shape(inputs)
            x = inputs
            seed = self.seed
            keep_prob = 1. - self.rate
            if seed is None:
                seed = np.random.randint(10e6)
            # the dummy 1. works around a TF bug
            # (float32_ref vs. float32 incompatibility)
            x= x*1
            name = None
            with ops.name_scope(name, "dropout", [x]) as name:
                x = ops.convert_to_tensor(x, name="x")
                if not x.dtype.is_floating:
                    raise ValueError("x has to be a floating point tensor since it's going to"
                       " be scaled. Got a %s tensor instead." % x.dtype)
                if isinstance(keep_prob, numbers.Real) and not 0 < keep_prob <= 1:
                    raise ValueError("keep_prob must be a scalar tensor or a float in the "
                       "range (0, 1], got %g" % keep_prob)
                keep_prob = ops.convert_to_tensor(
                             keep_prob, dtype=x.dtype, name="keep_prob")
                keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

                # Do nothing if we know keep_prob == 1
                if tensor_util.constant_value(keep_prob) == 1:
                    return x

                noise_shape = _get_noise_shape(x, noise_shape)
                # uniform [keep_prob, 1.0 + keep_prob)
                random_tensor = keep_prob
                random_tensor += random_ops.random_uniform(
                noise_shape, seed=seed, dtype=x.dtype)
               
                # 0. if [keep_prob, 1.0) and 1. if [1.0, 1.0 + keep_prob)
                binary_tensor = math_ops.floor(random_tensor)
                dim = binary_tensor.shape[2]//2

                symmetric_binary= tf.concat([binary_tensor[:,:,dim:],binary_tensor[:,:,dim:][::,::-1,::-1]], 2)
                ret = math_ops.div(x, keep_prob) * symmetric_binary
                
                return ret


    def get_config(self):
        config = {'rate': self.rate,
                  'noise_shape': self.noise_shape,
                  'seed': self.seed}
        base_config = super(MCRCDropout, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape


In [36]:
from keras.initializers import Initializer
def _compute_fans(shape, data_format='channels_last'):
    """Computes the number of input and output units for a weight shape.
    # Arguments
        shape: Integer shape tuple.
        data_format: Image data format to use for convolution kernels.
            Note that all kernels in Keras are standardized on the
            `channels_last` ordering (even when inputs are set
            to `channels_first`).
    # Returns
        A tuple of scalars, `(fan_in, fan_out)`.
    # Raises
        ValueError: in case of invalid `data_format` argument.
    """
    if len(shape) == 2:
        fan_in = shape[0]
        fan_out = shape[1]
    elif len(shape) in {3, 4, 5}:
        # Assuming convolution kernels (1D, 2D or 3D).
        # TH kernel shape: (depth, input_depth, ...)
        # TF kernel shape: (..., input_depth, depth)
        if data_format == 'channels_first':
            receptive_field_size = np.prod(shape[2:])
            fan_in = shape[1] * receptive_field_size
            fan_out = shape[0] * receptive_field_size
        elif data_format == 'channels_last':
            receptive_field_size = np.prod(shape[:-2])
            fan_in = shape[-2] * receptive_field_size
            fan_out = shape[-1] * receptive_field_size
        else:
            raise ValueError('Invalid data_format: ' + data_format)
    else:
        # No specific assumptions.
        fan_in = np.sqrt(np.prod(shape))
        fan_out = np.sqrt(np.prod(shape))
    return fan_in, fan_out

class RevcompVarianceScaling(Initializer):
    def __init__(self, scale=1.0,
                 mode='fan_in',
                 distribution='normal',
                 seed=None):
        if scale <= 0.:
            raise ValueError('`scale` must be a positive float. Got:', scale)
        mode = mode.lower()
        if mode not in {'fan_in', 'fan_out', 'fan_avg'}:
            raise ValueError('Invalid `mode` argument: '
                             'expected on of {"fan_in", "fan_out", "fan_avg"} '
                             'but got', mode)
        distribution = distribution.lower()
        if distribution not in {'normal', 'uniform'}:
            raise ValueError('Invalid `distribution` argument: '
                             'expected one of {"normal", "uniform"} '
                             'but got', distribution)
        self.scale = scale
        self.mode = mode
        self.distribution = distribution
        self.seed = seed

    def __call__(self, shape, dtype=None):
        fan_in, fan_out = _compute_fans(shape)
        fan_out = fan_out*2 #revcomp kernel underestimates fan_out
        print("fanin:",fan_in, "fanout:",fan_out, self.scale, self.mode)
        scale = self.scale
        if self.mode == 'fan_in':
            scale /= max(1., fan_in)
        elif self.mode == 'fan_out':
            scale /= max(1., fan_out)
        else:
            scale /= max(1., float(fan_in + fan_out) / 2)
        if self.distribution == 'normal':
            # 0.879... = scipy.stats.truncnorm.std(a=-2, b=2, loc=0., scale=1.)
            stddev = np.sqrt(scale) / .87962566103423978
            return K.truncated_normal(shape, 0., stddev,
                                      dtype=dtype, seed=self.seed)
        else:
            limit = np.sqrt(3. * scale)
            return K.random_uniform(shape, -limit, limit,
                                    dtype=dtype, seed=self.seed)

    def get_config(self):
        return {
            'scale': self.scale,
            'mode': self.mode,
            'distribution': self.distribution,
            'seed': self.seed
        }

# Reverse Complement

## Standard

In [31]:
scale = 1.0
rc_model_standard = keras.models.Sequential()
rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=x_train.shape[1:], padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard.add(k1.core.Activation("relu"))
rc_model_standard.add(RevCompSpatialDropout1D(0.5))
rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard.add(k1.core.Activation("relu"))
rc_model_standard.add(RevCompSpatialDropout1D(0.5))
rc_model_standard.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard.add(k1.core.Activation("relu"))
rc_model_standard.add(RevCompSumPool())
rc_model_standard.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard.add(Flatten())
rc_model_standard.add(RevCompSpatialDropout1D(0.5))
rc_model_standard.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard.add(keras_genomics.layers.core.Dense(units = 2))
rc_model_standard.add(k1.core.Activation("sigmoid"))

W0715 15:00:34.770263 139651693975296 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0715 15:00:34.781090 139651693975296 deprecation.py:506] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0715 15:00:34.882105 139651693975296 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [31]:
rc_model_standard.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_rc_standard = rc_model_standard.fit(x_train, noisy_y_train, validation_split=0.2,  
                    callbacks= [early_stopping_callback], batch_size=100,  epochs=100)
rc_model_standard.set_weights(early_stopping_callback.best_weights)


rc_standard_filename = ('rc_standard_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard.save(rc_standard_filename)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D, 
                  'RevCompSumPool':RevCompSumPool}
rc_standard_model_final = load_model(rc_standard_filename, custom_objects)

W0710 12:57:01.509592 140639318025984 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0710 12:57:01.533013 140639318025984 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0710 12:57:01.537446 140639318025984 deprecation.py:323] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0710 12:57:01.934440 140639318025984 deprecation_wrapper.py:119] From /users/hannahgz/anaconda3/lib/python3.7/site-packages/keras/backend/tensor

Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 9s 759us/step - loss: 0.7261 - acc: 0.4999 - val_loss: 0.6928 - val_acc: 0.5143
Epoch 2/100
12000/12000 [==============================] - 4s 340us/step - loss: 0.6931 - acc: 0.5060 - val_loss: 0.6927 - val_acc: 0.5145
Epoch 3/100
12000/12000 [==============================] - 4s 340us/step - loss: 0.6931 - acc: 0.5087 - val_loss: 0.6928 - val_acc: 0.5172
Epoch 4/100
12000/12000 [==============================] - 4s 364us/step - loss: 0.6930 - acc: 0.5045 - val_loss: 0.6929 - val_acc: 0.5125
Epoch 5/100
12000/12000 [==============================] - 4s 368us/step - loss: 0.6930 - acc: 0.5065 - val_loss: 0.6929 - val_acc: 0.5112
Epoch 6/100
12000/12000 [==============================] - 4s 350us/step - loss: 0.6928 - acc: 0.5122 - val_loss: 0.6921 - val_acc: 0.5192
Epoch 7/100
12000/12000 [==============================] - 4s 362us/step - loss: 0.6918 - acc: 0.5189 - val_loss: 0.6

In [45]:
scale = 1.0
rc_model_standard_mcdropout = keras.models.Sequential()
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=x_train.shape[1:], padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(RevCompSumPool())
rc_model_standard_mcdropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard_mcdropout.add(Flatten())
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard_mcdropout.add(keras_genomics.layers.core.Dense(units = 2))
rc_model_standard_mcdropout.add(k1.core.Activation("sigmoid"))

IndexError: list index out of range

# Regular Model

In [43]:
reg_model = keras.models.Sequential()
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=(input_length,4), padding="same"))
# reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Dropout(0.2))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
# reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Dropout(0.2))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
# reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Dropout(0.2))
reg_model.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
reg_model.add(Flatten())
reg_model.add(k1.Dense(units = 100, activation = "relu"))
reg_model.add(k1.Dropout(0.2))
reg_model.add(k1.Dense(units = 2))
reg_model.add(k1.core.Activation("sigmoid"))

In [37]:
reg_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_reg = reg_model.fit(x_train, noisy_y_train, validation_split = 0.2, 
                           callbacks=[early_stopping_callback], batch_size = 100, epochs = 100)

Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 3s 260us/step - loss: 0.6945 - acc: 0.4987 - val_loss: 0.6932 - val_acc: 0.4917
Epoch 2/100
12000/12000 [==============================] - 2s 176us/step - loss: 0.6928 - acc: 0.5155 - val_loss: 0.6930 - val_acc: 0.5183
Epoch 3/100
12000/12000 [==============================] - 2s 177us/step - loss: 0.6798 - acc: 0.5580 - val_loss: 0.6414 - val_acc: 0.6322
Epoch 4/100
12000/12000 [==============================] - 2s 176us/step - loss: 0.6394 - acc: 0.6193 - val_loss: 0.6234 - val_acc: 0.6383
Epoch 5/100
12000/12000 [==============================] - 2s 173us/step - loss: 0.6208 - acc: 0.6455 - val_loss: 0.6142 - val_acc: 0.6473
Epoch 6/100
12000/12000 [==============================] - 2s 175us/step - loss: 0.6100 - acc: 0.6595 - val_loss: 0.6079 - val_acc: 0.6718
Epoch 7/100
12000/12000 [==============================] - 2s 177us/step - loss: 0.5922 - acc: 0.6919 - val_loss: 0.5

In [38]:
reg_model.set_weights(early_stopping_callback.best_weights)
reg_filename = ('reg_%s.h5' % seed_num, str(seed_num))[0]
reg_model.save(reg_filename)
reg_model_final = load_model(reg_filename)

## Augmented

In [39]:
augment_model = keras.models.Sequential()
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=(input_length,4), padding="same"))
# reg_model.add(k1.BatchNormalization())
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
# reg_model.add(k1.BatchNormalization())
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
# reg_model.add(k1.BatchNormalization())
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
augment_model.add(Flatten())
augment_model.add(k1.Dense(units = 100, activation = "relu"))
augment_model.add(k1.Dense(units = 2))
augment_model.add(k1.core.Activation("sigmoid"))

In [40]:
augment_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_augment = augment_model.fit(x_train_augment, noisy_y_train_augment, validation_split = 0.2, 
                           callbacks=[early_stopping_callback], batch_size = 200, epochs = 100)

Train on 24000 samples, validate on 6000 samples
Epoch 1/100
24000/24000 [==============================] - 6s 245us/step - loss: 0.6937 - acc: 0.5058 - val_loss: 0.6929 - val_acc: 0.5097
Epoch 2/100
24000/24000 [==============================] - 3s 138us/step - loss: 0.6931 - acc: 0.5051 - val_loss: 0.6928 - val_acc: 0.5137
Epoch 3/100
24000/24000 [==============================] - 3s 141us/step - loss: 0.6735 - acc: 0.5601 - val_loss: 0.6263 - val_acc: 0.6387
Epoch 4/100
24000/24000 [==============================] - 3s 142us/step - loss: 0.6250 - acc: 0.6423 - val_loss: 0.6094 - val_acc: 0.6602
Epoch 5/100
24000/24000 [==============================] - 3s 140us/step - loss: 0.5892 - acc: 0.7054 - val_loss: 0.5717 - val_acc: 0.7206
Epoch 6/100
24000/24000 [==============================] - 4s 154us/step - loss: 0.5699 - acc: 0.7313 - val_loss: 0.5539 - val_acc: 0.7530
Epoch 7/100
24000/24000 [==============================] - 4s 146us/step - loss: 0.5579 - acc: 0.7425 - val_loss: 0.5

In [41]:
augment_model.set_weights(early_stopping_callback.best_weights)
augment_filename = ('augment_%s.h5' % seed_num, str(seed_num))[0]
augment_model.save(augment_filename)
augment_final = load_model(augment_filename)

# Siamese Architecture

In [42]:
s_model = Sequential([
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
            input_shape=(input_length,4), padding="same"), 
#     k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
#     k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
#     k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40), 
    Flatten(), 
    k1.Dense(units = 100, activation = "relu"),
    k1.Dense(units = 2)
], name = "shared_layers")

main_input = Input(shape=(input_length, 4, ))
rev_input = Input(shape=(input_length, 4, ))

main_output = s_model(main_input)
rev_output = s_model(rev_input)

avg = k1.Average()([main_output, rev_output])
final_out = k1.core.Activation("sigmoid")(avg)
siamese_model = Model(inputs = [main_input, rev_input], outputs = final_out)

merged = keras.layers.concatenate([main_output, rev_output])

predictions = k1.core.Activation("sigmoid")

In [43]:
siamese_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
siamese_model.fit([x_train, x_train[:,::-1,::-1]], noisy_y_train, validation_split=0.2,  
                    callbacks=[early_stopping_callback],
                    batch_size=100,  epochs=100)

Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 5s 413us/step - loss: 0.6943 - acc: 0.4996 - val_loss: 0.6931 - val_acc: 0.5098
Epoch 2/100
12000/12000 [==============================] - 4s 303us/step - loss: 0.6932 - acc: 0.5054 - val_loss: 0.6926 - val_acc: 0.5135
Epoch 3/100
12000/12000 [==============================] - 4s 293us/step - loss: 0.6885 - acc: 0.5224 - val_loss: 0.6518 - val_acc: 0.6038
Epoch 4/100
12000/12000 [==============================] - 4s 300us/step - loss: 0.6359 - acc: 0.6205 - val_loss: 0.6147 - val_acc: 0.6423
Epoch 5/100
12000/12000 [==============================] - 4s 294us/step - loss: 0.6185 - acc: 0.6363 - val_loss: 0.6132 - val_acc: 0.6228
Epoch 6/100
12000/12000 [==============================] - 4s 292us/step - loss: 0.6124 - acc: 0.6402 - val_loss: 0.6105 - val_acc: 0.6402
Epoch 7/100
12000/12000 [==============================] - 3s 286us/step - loss: 0.6093 - acc: 0.6460 - val_loss: 0.6

In [44]:
siamese_model.set_weights(early_stopping_callback.best_weights)
siamese_filename = ('siamese_%s.h5' % seed_num, str(seed_num))[0]
siamese_model.save(siamese_filename)
siamese_model_final = load_model(siamese_filename)

# Evaluating Models

In [45]:
print("Reverse Complement: Standard")
f.write("Reverse Complement: Standard\n")
evaluate(rc_model_standard, x_test, y_test)

Reverse Complement: Standard
auroc: 0.7583246454570032
auprc: 0.7675953310940138


In [46]:
print("Reverse Complement: Variance")
f.write("Reverse Complement: Variance\n")
evaluate(rc_model_var, x_test, y_test)

Reverse Complement: Variance
auroc: 0.7511103892287384
auprc: 0.7634567804093357


In [47]:
print("Regular Model")
f.write("Regular Model\n")
evaluate(reg_model, x_test, y_test)

Regular Model
auroc: 0.9347747084022962
auprc: 0.9442786115200366


In [1]:
print("Augmented Model")
f.write("Augment Model\n")
evaluate(augment_model, x_test_augment, y_test_augment)

Augmented Model


NameError: name 'f' is not defined

In [49]:
print("Siamese Architecture")
y_pred = siamese_model.predict([x_test, x_test[:,::-1,::-1]])
auroc = roc_auc_score(y_test, y_pred)
auprc = average_precision_score(y_test, y_pred)
print("auroc: " + str(auroc))
print("auprc: "+ str(auprc))
f.write("Siamese Architecture\n")
f.write("auroc: " + str(auroc) + "\n")
f.write("auprc: "+ str(auprc) + "\n")

Siamese Architecture
auroc: 0.7541213656987258
auprc: 0.7653844345521915


26

In [50]:
f.close()